In [1]:
# model_name = 'GoToCompany/gemma2-9b-cpt-sahabatai-v1-base'
model_name = "google/gemma-2-9b-it"
from unsloth import FastLanguageModel
test_language = 'su'
import os
import re
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 8192,
    load_in_4bit = True,
    # token = "hf_..."
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-21 12:25:27.108904: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747815927.124417 1783607 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747815927.129288 1783607 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747815927.141888 1783607 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747815927.141901 1783607 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747815927.141903 1783607 computation_placer.cc:177] computation placer alr

🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2025.5.6: Fast Gemma2 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.64 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.1+cu121. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.0.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584, padding_idx=0)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=3584, bias=False)
          (rotary_emb): GemmaFixedRotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=3584, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (post_attention_laye

In [2]:
import pandas as pd

data_path = f"../../dataset/test/test_{test_language}.csv"
def load_test_data(data_path):
    data = pd.read_csv(data_path)
    contexts = []
    endings = []
    EOS_TOKEN = tokenizer.eos_token
    for idx, row in data.iterrows():
        sents = []
        for i in [4,3,2,1]:
            current_sentence = row[f'sentence_{i}']
            current_sentence = current_sentence + '.' if current_sentence[-1] != '.' else current_sentence
            sents.insert(0, current_sentence)
        context = ' '.join(sents) 
        ending = row['correct_ending']
        # ending2 = row['incorrect_ending']
        contexts.append(context)
        endings.append(ending)

    # convert to huggingface dataset
    from datasets import Dataset
    dataset = Dataset.from_dict({'context': contexts, 'ending': endings})
    def formatting_prompts_func(examples):
        contexts  = examples["context"]
        endings = examples["ending"]
        texts = []
        for context, ending in zip(contexts, endings):
            chat = [
                {"role": "user", "content": f"Generate the ending for the following given story context\nStory Context: {context}\nProvide the ending in a single sentence. Don't add anything else."},
            ]
            text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
            # text = alpaca_prompt.format(context)
            texts.append(text)
        return { "text" : texts, }
    dataset = dataset.map(formatting_prompts_func, batched = True,)
    return dataset
dataset = load_test_data(data_path)
contexts = dataset['text']
correct_endings = dataset['ending']

Map:   0%|          | 0/1094 [00:00<?, ? examples/s]

In [7]:
from tqdm.auto import tqdm
import torch

context = contexts[]

inputs = tokenizer(batch_contexts, return_tensors="pt", truncation=True).to("cuda")

all_predictions = []

for i in tqdm(range(0, len(batch_contexts), batch_size)):
    input_ids_batch = inputs['input_ids'][i:i+batch_size]
    attention_mask_batch = inputs['attention_mask'][i:i+batch_size]
    
    with torch.no_grad():
        res = model.generate(
            input_ids=input_ids_batch,
            attention_mask=attention_mask_batch,
            max_new_tokens=128
        )
    
    # Decode only the generated part
    for j, output in enumerate(res):
        input_len = (input_ids_batch[j] != tokenizer.pad_token_id).sum()
        generated = output[input_len:]
    decoded = tokenizer.decode(generated, skip_special_tokens=True).strip()
    all_predictions.append(decoded)

# `all_predictions` now contains the generated endings for each context


  0%|          | 0/4 [00:00<?, ?it/s]

AUTOTUNE bmm(256x146x256, 256x256x146)
  triton_bmm_45 0.0727 ms 100.0%
  triton_bmm_39 0.0737 ms 98.6%
  triton_bmm_44 0.0737 ms 98.6%
  triton_bmm_51 0.0748 ms 97.3%
  triton_bmm_40 0.0748 ms 97.2%
  triton_bmm_47 0.0756 ms 96.1%
  triton_bmm_41 0.0758 ms 95.9%
  triton_bmm_49 0.0758 ms 95.9%
  bmm 0.0768 ms 94.7%
  triton_bmm_48 0.0778 ms 93.4%
SingleProcess AUTOTUNE benchmarking takes 1.9436 seconds and 0.0077 seconds precompiling
AUTOTUNE bmm(256x146x146, 256x146x256)
  triton_bmm_66 0.0696 ms 100.0%
  triton_bmm_73 0.0748 ms 93.2%
  triton_bmm_68 0.0758 ms 91.9%
  triton_bmm_67 0.0767 ms 90.8%
  triton_bmm_59 0.0778 ms 89.5%
  triton_bmm_63 0.0778 ms 89.5%
  triton_bmm_62 0.0809 ms 86.1%
  triton_bmm_64 0.0850 ms 81.9%
  triton_bmm_70 0.0850 ms 81.9%
  triton_bmm_71 0.0870 ms 80.0%
SingleProcess AUTOTUNE benchmarking takes 2.0481 seconds and 0.0011 seconds precompiling
/home/rifo.genadi/.conda/envs/local_cloze/lib/python3.10/site-packages/unsloth/kernels/utils.py:443: UserWarning

# Compute BertScore

In [10]:
import

tensor([     2,      2,    106,   1645,    108,  38557,    573,  16851,    604,
           573,   2412,   2764,   3904,   4807,    108,  21519,  19422, 235292,
         37747, 235259,   1957,   1063,   4221,    569,   1465,    553,  20953,
        114317,   1966,   1569, 235248, 235284, 235308,   3586,    549, 235265,
          4378,   1569, 235248, 235284, 235304,   3586,    549,  60068,   1021,
        235269,    625,  11055,    798,  40471,    682,   4280,    575,   9732,
         31678,   1577,  40608, 235250, 235265,    584,  47405,  99403,   8550,
           966, 235269,  31678,   1577,  59764,  51730,    655,   1021,    575,
          9732,    556,    693,   2998, 235265,   1881,   1063,    556,    493,
           541,  59764,   8227, 148152, 211157,  99403,   5675,   2624, 235259,
        144921, 207099,    699,   3068, 235265,    108,  70029,    573,  16851,
           575,    476,   3821,  13060, 235265,   4257, 235303, 235251,   1843,
          4341,   1354, 235265,    107, 